In [146]:
%pylab inline

import requests
import eventlet
import pandas as pd
import operator
#pip install mglearn
import mglearn

from bs4 import BeautifulSoup
from sklearn.decomposition import PCA
from IMDBAPI import IMDB
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import LatentDirichletAllocation

imdb = IMDB()

################ <Functions> ################# 
def plotcounts (a):
    temp=a
    temp.replace(".", " ").replace(",", "")
    temp=temp.split(" ")
    temp=map(lambda x:x.lower(),temp)
    words=[]
    for b in movie_words:
        if b in temp:
            words.append(b)
    return words

def worddummy (a):
    if m in a:
        return 1
    else:
        return 0

def countactors(a):
    temp.append(a.split(", "))
    return temp

def token(text):
    return(text.split("|"))

################ <IMDB films extractor> #################
URL = "http://www.imdb.com/chart/top"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html.parser')
entries=soup.findAll('div', class_="wlb_ribbon")
# getting top movies ids from imdb
movie_ids = []
for a in entries:
    movie_ids.append(a['data-tconst'])
#print("========= movie_ids:", movie_ids) 
         
header = 'http://www.omdbapi.com/?apikey=6be019fc&tomatoes=true&i='
movie_info = []
for i in movie_ids:
    url = header + i
#print("========= url:", url) 
    r = requests.get(url).json()
    movie = []
    for a in r.keys():
        movie.append(r[a])
    movie_info.append(movie)
columns = r.keys()
df = pd.DataFrame(movie_info, columns = columns)
#print("========= df:", df)

################ <Extrating from films plot the most valuable words> #################
plots = list(df['Plot'])
temp = ""
for p in plots:
        temp = temp + p
temp.replace(".", " ").replace(",", "")
temp = temp.split(" ")
temp = list(map(lambda x:x.lower(),temp))
#print("=== plot_temp:", temp)

# collect valuable for further groups generatig words
vect = CountVectorizer(max_features=10000, max_df=.15, stop_words="english").fit(temp)
X = vect.transform(temp)

# test result
feature_names = vect.get_feature_names()
# print("=== Number of features: {}".format(len(feature_names)))
# print("=== The first 20 features:\n{}".format(feature_names[:20]))
# print("=== Each 2000 feature:\n{}".format(feature_names[::2000]))

# Max and min groups number (if number is greater than MAX_GROUPS_NUMBER - groups won`t be dense, info become useless)
# Here is LDA (Latent Dirichlet allocation - Латентное размещение Дирихле) is used for clustering (building topic model)
MAX_GROUPS_NUMBER = 85
#MIN_GROUPS_NUMBER = 10
lda = LatentDirichletAllocation(n_components=MAX_GROUPS_NUMBER, learning_method="batch", max_iter=25, random_state=0)
document_topics = lda.fit_transform(X)

# Sorting of features acsending order 
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# Getting of features names
feature_names = np.array(vect.get_feature_names())
# Take some number of topics to view results
mglearn.tools.print_topics(topics=range(20), feature_names=feature_names,
 sorting=sorting, topics_per_chunk=5, n_words=10)

################ <Demo: taking films from first two groups> ################# 
# (!!!) Take into account: films duplications were removed
# sorting by weight document of the topic number 1
group1 = np.argsort(document_topics[:, 1])[::-1]
films_1 = []

for i in group1[:10]:
    films_1.append(df[df['Plot'].str.contains(temp[i])].Title.unique())

print("========= First cluster\n")
for i in set(map(tuple, films_1)):
    print(''.join(list(i)))
    
# sorting by weight document of the topic number 2
group2 = np.argsort(document_topics[:, 2])[::-1]
films_2 = []

for i in group2[:10]:
    films_2.append(df[df['Plot'].str.contains(temp[i])].Title.unique())

print("========= Second cluster\n")
for i in set(map(tuple, films_2)):
    print('\n'.join(list(i)))
    
# sorting by weight document of the topic number 3
# group3 = np.argsort(document_topics[:, 3])[::-1]
# films_3 = []

# for i in group3[:10]:
#     films_3.append(df[df['Plot'].str.contains(temp[i])].Title.unique())

# print("========= Third cluster\n")
# for i in set(map(tuple, films_3)):
#     print('\n', list(i))
    
# and so on ...
# printing will be optimized in one method

Populating the interactive namespace from numpy and matplotlib
topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
jewish        gets          local         teacher       middle        
men           weed          bride         army          turn          
english       union         assassinate   singer        puts          
fortune       growers       college       zubrowka      make          
grande        indulgent     trouble       firing        brother       
sound         self          star          flesh         earth         
flesh         boyhood       obstacles     flees         century       
flees         tutsi         surprise      flee          self          
flee          firepower     maker         flame         conscious     
flame         oskar         investigator  firepower     awakening     


topic 5       topic 6       topic 7       topic 8       topic 9       
--------    